In [2]:
'''
Created on : April 5

source : ReferencePaperForACO.pdf

'''

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('Bank_Personal_Loan_Modelling.csv')

X = df.drop('Personal Loan', axis=1)
y = df['Personal Loan']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [3]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(32, input_dim=12, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


2023-04-05 09:36:21.095700: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-05 09:36:21.785657: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-05 09:36:21.843989: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-05 09:36:21.844012: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import numpy as np

scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.transform(X_test)

n_ants = 50
n_iterations = 100
alpha = 1
beta = 1
rho = 0.1

pheromone = np.random.uniform(low=-1, high=1, size=model.get_weights().__len__())

for i in range(n_iterations):
    ants = np.random.choice(n_ants, size=n_ants, replace=True)
    solutions = []
    for ant in ants:
        model.set_weights(pheromone)
        model.fit(X_train_norm, y_train, epochs=10, verbose=0)
        
        y_pred = model.predict(X_test_norm)
        y_pred = (y_pred > 0.5).astype(int)
        accuracy = accuracy_score(y_test, y_pred)
        
        solutions.append((accuracy, model.get_weights()))
    
    solutions.sort(reverse=True)
    
    pheromone *= (1 - rho)
    for j in range(len(solutions)):
        weights = solutions[j][1]
        for k in range(len(weights)):
            pheromone[k] += alpha * solutions[j][0] * weights[k]
    
    model.set_weights(pheromone)

y_pred = model.predict(X_test_norm)
y_pred = (y_pred > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
